In [10]:
import os
import re
from datetime import datetime
from pathlib import Path
import datetime
import folium
import shapely
import json
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
import geopandas as gpd

In [11]:
DATA_PATH = Path('/home/ioannisgkanatsios/Documents/sentinel1_download')

### add username and password

In [12]:
user = os.environ['user']
password = os.environ['password']
api_url = "https://scihub.copernicus.eu/dhus/"

In [13]:
# Sentinel-1 attributes
s1_attr = {'product':{'S':'SLC', 'G':'GRD','O':'OCN'},
           'orbit':{'A':'ascending', 'D':'descending'}
          }

### search for scenes

In [14]:
# search by polygon in Geojson format
footprint = geojson_to_wkt(read_geojson(DATA_PATH.joinpath('footprint','aoi.geojson')))
# api query
api = SentinelAPI(
    user=user,
    password=password,
    api_url= api_url 
)
# perform query based on date
products = api.query(footprint,
                     date=('20190201', '20190316'), 
                     platformname='Sentinel-1',
                     producttype=s1_attr['product']['G'],
                     orbitdirection=s1_attr['orbit']['A'],
                     #relativeOrbitNumber=164
                    )
print (f'{len(products)} scenes have been found')
    

20 scenes have been found


In [7]:
# convert to Pandas DataFrame
products_df = api.to_geodataframe(products)
products_df[:3]

/home/ioannisgkanatsios/.local/share/virtualenvs/sentinel1_download-ggW_R6lz/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,title,link,link_alternative,link_icon,summary,ondemand,beginposition,endposition,ingestiondate,missiondatatakeid,slicenumber,orbitnumber,lastorbitnumber,relativeorbitnumber,lastrelativeorbitnumber,sensoroperationalmode,swathidentifier,orbitdirection,producttype,timeliness,platformname,platformidentifier,instrumentname,instrumentshortname,filename,format,productclass,polarisationmode,acquisitiontype,status,size,identifier,uuid,geometry
3c1786d1-7d9b-484c-a505-b5ddf61e1f36,S1A_IW_GRDH_1SDV_20190315T175752_20190315T1758...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-03-15T17:57:52.163Z, Instrument: SA...",false,2019-03-15 17:57:52.163,2019-03-15 17:58:17.162,2019-03-15 21:28:50.519,193153,13,26352,26352,30,30,IW,IW,ASCENDING,GRD,Fast-24h,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S1A_IW_GRDH_1SDV_20190315T175752_20190315T1758...,SAFE,S,VV VH,NOMINAL,ARCHIVED,1.63 GB,S1A_IW_GRDH_1SDV_20190315T175752_20190315T1758...,3c1786d1-7d9b-484c-a505-b5ddf61e1f36,"MULTIPOLYGON (((-4.18945 51.18417, -0.48280 51..."
586316e7-2deb-4e84-a660-d18b058b7b99,S1A_IW_GRDH_1SDV_20190315T175727_20190315T1757...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-03-15T17:57:27.164Z, Instrument: SA...",false,2019-03-15 17:57:27.164,2019-03-15 17:57:52.162,2019-03-15 21:28:44.862,193153,12,26352,26352,30,30,IW,IW,ASCENDING,GRD,Fast-24h,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S1A_IW_GRDH_1SDV_20190315T175727_20190315T1757...,SAFE,S,VV VH,NOMINAL,ARCHIVED,1.63 GB,S1A_IW_GRDH_1SDV_20190315T175727_20190315T1757...,586316e7-2deb-4e84-a660-d18b058b7b99,"MULTIPOLYGON (((-3.71279 49.69001, -0.12286 50..."
1ad3d196-6593-4728-983b-c6e49abd8edd,S1A_IW_GRDH_1SDV_20190310T174911_20190310T1749...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-03-10T17:49:11.228Z, Instrument: SA...",false,2019-03-10 17:49:11.228,2019-03-10 17:49:40.254,2019-03-10 21:10:37.336,192471,1,26279,26279,132,132,IW,IW,ASCENDING,GRD,Fast-24h,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S1A_IW_GRDH_1SDV_20190310T174911_20190310T1749...,SAFE,S,VV VH,NOMINAL,ARCHIVED,1.85 GB,S1A_IW_GRDH_1SDV_20190310T174911_20190310T1749...,1ad3d196-6593-4728-983b-c6e49abd8edd,"MULTIPOLYGON (((-1.55459 49.56566, 0.00000 49...."


### Product metadata

In [7]:
uuid = products_df['uuid'][0]
product_meta = api.get_product_odata(uuid, full=True)

In [8]:
product_meta

{'id': '40a91bef-c25a-4d32-b826-443b4788e70b',
 'title': 'S1A_IW_GRDH_1SDV_20190813T174947_20190813T175012_028554_033AAC_154B',
 'size': 1000039314,
 'md5': '9CCB572D78F467DD676625D8CE68A58B',
 'date': datetime.datetime(2019, 8, 13, 17, 49, 47, 827000),
 'footprint': 'POLYGON((-2.601100 52.793621,1.157206 53.194427,1.532326 51.699444,-2.102490 51.301105,-2.601100 52.793621))',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('40a91bef-c25a-4d32-b826-443b4788e70b')/$value",
 'Online': False,
 'Creation Date': datetime.datetime(2019, 8, 13, 20, 43, 8, 640000),
 'Ingestion Date': datetime.datetime(2019, 8, 13, 20, 42, 58, 873000),
 'Acquisition Type': 'NOMINAL',
 'Carrier rocket': 'Soyuz',
 'Cycle number': 177,
 'Date': datetime.datetime(2019, 8, 13, 17, 49, 47, 827000),
 'Filename': 'S1A_IW_GRDH_1SDV_20190813T174947_20190813T175012_028554_033AAC_154B.SAFE',
 'Footprint': '<gml:Polygon srsName="http://www.opengis.net/gml/srs/epsg.xml#4326" xmlns:gml="http://www.opengis.net/gml

### Folium - add data on the map

In [15]:
aoi = DATA_PATH.joinpath('footprint','aoi.geojson')

In [16]:
gdf = gpd.read_file(aoi)

#### Convert shapefile to Geojson

In [17]:
# FIX the error: datetime.datetime(...) is not JSON serializable
# https://code-maven.com/serialize-datetime-object-as-json-in-python
def myconverter(o):
    if isinstance(o, datetime.datetime):
        return o.__str__()
    
scenes = json.dumps(shapely.geometry.mapping(products_df), default = myconverter)

In [18]:
# Locate AOI on the map
location = [52.8, -1.4]
m = folium.Map(location=location,
               zoom_start=5,
               name='Basemap'
              )

# overlay Sentinel-1 scenes
folium.GeoJson(
    scenes,
    name='Sentinel-1 scenes',
    # polygon styling: https://gist.github.com/wrobstory/5609786
    style_function= lambda x :{'fillColor':'grey', 
                               'fillOpacity': 0.3, 
                               'color':'black',
                               'weight': 1
                              }
).add_to(m)


# overlay our AOI
folium.GeoJson(
    gdf,
    name='AOI',
).add_to(m)


folium.map.LayerControl('topright', 
                        collapsed=False
).add_to(m)
m

### Check if the prodcut is online or a [Long Term Archive (LTA)](https://scihub.copernicus.eu/userguide/LongTermArchive)
- Download products acquired in specific days

In [8]:
for i, row in products_df.iterrows():
    product = row[['title','uuid','beginposition']]
    # convert datatime to string
    date = product['beginposition']
    date_str = date.strftime('%Y-%m-%d')
    uuid = product['uuid']
    product_meta = api.get_product_odata(uuid)
    if product_meta['Online']:
        print('Product {} is online.'.format(product_meta['title']))
        # if online, download it
        api.download(product['uuid'], DATA_PATH.joinpath('output'))
    else:
        print('Product {} is not online.'.format(product_meta['title']))
        

Product S1A_IW_GRDH_1SDV_20190315T175752_20190315T175817_026352_02F281_1685 is not online.
Product S1A_IW_GRDH_1SDV_20190315T175727_20190315T175752_026352_02F281_A0D7 is not online.
Product S1A_IW_GRDH_1SDV_20190310T174911_20190310T174940_026279_02EFD7_1456 is not online.
Product S1A_IW_GRDH_1SDV_20190310T174940_20190310T175005_026279_02EFD7_6D76 is not online.
Product S1B_IW_GRDH_1SDV_20190309T175659_20190309T175724_015281_01C988_0DFC is not online.
Product S1B_IW_GRDH_1SDV_20190304T174852_20190304T174917_015208_01C731_A890 is not online.
Product S1B_IW_GRDH_1SDV_20190304T174827_20190304T174852_015208_01C731_E0A3 is not online.
Product S1A_IW_GRDH_1SDV_20190226T174911_20190226T174940_026104_02E98B_516D is not online.
Product S1A_IW_GRDH_1SDV_20190226T174940_20190226T175005_026104_02E98B_D7CA is not online.
Product S1A_IW_GRDH_1SDV_20190219T175752_20190219T175817_026002_02E5D1_4967 is not online.
Product S1A_IW_GRDH_1SDV_20190219T175727_20190219T175752_026002_02E5D1_643F is not online.